In [20]:
import time
import numpy as np
import pandas as pd

from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


# Controlling the webdriver  
Firstly, we want to access the webpage doctolib. To do that we have to create a connector with Selenium and the launch the driver

In [21]:
tempsAttente = 3    #temps d'attente en secondes avant qu'il y a un timeout de Webdriverwait.until()

class Driver:

    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        options.add_argument("--disable-blink-features")
        options.add_argument("disable-popup-blocking")
        options.add_argument("disable-notifications")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        self.driver = webdriver.Chrome(ChromeDriverManager().install())


    #Lancer google chrome avec la page doctolib

    def connect(self, url):
        self.driver.get(url)
        # TabsInteraction.TabsCreation(self.driver, "https://www.doctolib.fr", "")
        # TabsInteraction.switchToTab(self.driver, 0)
        print("Connecting to", url)
        time.sleep(1)


    def getSpecializations(self):

        specializations = []        

        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')))
        text_area = self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')

        letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


        for letter in letters :
            text_area.clear()
            time.sleep(1)
            text_area.send_keys(letter)
            time.sleep(2)

            #getting the first result
            try:
                path = '//button[@class="searchbar-result searchbar-result-active"]'
                WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, path)))
                element = self.driver.find_element(By.XPATH, path)
                specialization = element.get_attribute('id')

                if specialization not in specializations:
                    specializations.append(specialization)
                else:
                    print('Specialization ', specialization, 'already found')
                    
                print(specializations[-1])

                #getting the other results
                try :
                    path = '//button[@class="searchbar-result"]'
                    WebDriverWait(self.driver, 2).until(EC.visibility_of_element_located((By.XPATH, path)))
                    elements = self.driver.find_elements(By.XPATH, path)
                    
                    for element in elements:
                        specializations.append(element.get_attribute('id'))
                        print(specializations[-1])

                except:
                    pass

            except:
                pass
        
        return specializations


    #for now we get only the first doctors that appear in the page
    def getDoctors(self, specialization, city):

        doctors = []

        #writing the specialization
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').clear()
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').send_keys(specialization)
        time.sleep(1)

        #writing the city
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').clear()
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').send_keys(city)
        time.sleep(1)
        #We need to enter the city to register it
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').send_keys(Keys.ENTER)

        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//span[@class="searchbar-submit-button-label dl-text-transform-none"]')))
        self.driver.find_element(By.XPATH, '//span[@class="searchbar-submit-button-label dl-text-transform-none"]').click()
        
        print('Searching results for :', specialization, 'in', city)
        time.sleep(2)
        
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-search-result"]')))
        docs = self.driver.find_elements(By.XPATH, '//div[@class="dl-search-result"]')
        
        for doctor in docs:
            #first we get the "easy" features
            name = doctor.find_element(By.XPATH, './/h3[@data-test="dl-search-result-name"]').text
            city = city
            specialization = specialization

            #the the harder ones
            try:
                WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, './/div[@class="Tappable-inactive availabilities-slot"]')))
                first_dispo = doctor.find_element(By.XPATH, './/div[@class="Tappable-inactive availabilities-slot"]').get_attribute('aria-label')
            except:
                first_dispo = 'null'    #means that there are no near in time disponibilites

            fee = ""
            doctors.append(Doctor(name, city, specialization, first_dispo, fee))
            
        return doctors

    def getFees(self, name):
        self.driver.find_element(By.XPATH, '//div[@class="logo-doctolib logo-doctolib-white"]').click()

        #writing the doctor name
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').clear()
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').send_keys(name)
        #We need to enter the city to register it
        try:
            WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input searchbar-query-input-opened"]')))
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input searchbar-query-input-opened"]').send_keys(Keys.ENTER)
        except:
            return 'Weird value'

        try:
            WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-profile-fee-tag"]')))
            fee = self.driver.find_element(By.XPATH, '//div[@class="dl-profile-fee-tag"]').text
            fee = str(int(fee.split(" ")[0]))
        except:
            fee = 'Conventionné'        #if the fee tag cannot be found it means that it is conventionné

        return fee
            
        


We created a Doctor class to manipulate more easily the doctors features

In [22]:
class Doctor():
    def __init__(self, name, city, specialization, first_dispo, fee):
        self.name = name
        self.city = city
        self.specialization = specialization
        self.first_dispo = first_dispo
        self.fee = fee

    def features(self):
        return(self.name, self.city, self.specialization, self.first_dispo, self.fee)

These are the cities and specializations we looked at first :  
Lyon and Paris are bug cities and Sancerre and Confolens are small ones, we did this in order to compare fees and disponibilites at first glance.

In [23]:
cities = ['Paris', 'Lyon', 'Sancerre', 'Confolens']
specializations = ['Medecin generaliste', 'Dentiste']

In [ ]:
connector = Driver()
connector.connect("https://www.doctolib.fr")
doctors = []

#Get all the doctors names, cities, specializations and their next possible reservation
for city in cities:
    for spec in specializations:
        doctors = doctors + connector.getDoctors(spec, city)

In [25]:
for doctor in doctors:
    if doctor.city == 'Paris':
        print(doctor.features())

('Dr Joël Valendoff', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Ridha MAHJOUB', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Léa Athias', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Patricia Mussuto', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Maurice KONQUI', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Didier Montloin', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Isaac Bohbot', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Serge ALPERIN', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Françoise BARRO-LECOMTE', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Eric FISCHER', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Jean-Luc THOMAS', 'Paris', 'Medecin generaliste', 'null', '')
('Institut Alfred Fournier', 'Paris', 'Medecin generaliste', 'null', '')
('Centre de Santé (CDS) - Bauchat Nation', 'Paris', 'Medecin generaliste', 'null', '')
('Centre de santé de Belleville', 'Paris', 'Medecin generaliste', 'null', '')
('Dr Marie Paule VIGLA', 'Paris',

We can see that there are no disponibilites for any doctor. This means that we will have to dig for more doctors as we are only taking the first ones at the moment.

In [ ]:
#Now we need to find the fees for each doctor

connector = Driver()
connector.connect("https://www.doctolib.fr")

for doctor in doctors:
    fee = connector.getFees(doctor.name)
    doctor.fee = fee
    

In [ ]:
doctors_features = []

for doctor in doctors:
    doctors_features.append(doctor.features())

In [ ]:
pd.DataFrame(doctors_features, columns=['Name', 'Specialization', 'City', 'Waiting Time', 'Fees']).to_csv("doctors_features.csv")